In [2]:
import tensorflow 
import h5py

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout
from tensorflow.keras.models import load_model

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout, concatenate, Input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

2021-09-03 15:58:53.826523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-03 15:58:53.826558: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
modelcamera=DenseNet121(weights='imagenet',include_top=False, input_shape=(32, 32, 3)) 

x1=modelcamera.output

x1= GlobalAveragePooling2D()(x1)
x1= BatchNormalization()(x1)
x1= Dropout(0.5)(x1)
x1= Dense(1024,activation='relu')(x1) 
x1= Dense(512,activation='relu')(x1) 
x1= BatchNormalization()(x1)
x1= Dropout(0.5)(x1)

# preds=Dense(8, activation='softmax')(x1) #FC-layer

In [ ]:
modelthermal=DenseNet121(weights='imagenet',include_top=False, input_shape=(32, 32, 3)) 
for i, layer in enumerate(modelthermal.layers):
    layer._name = 'layer_' + str(i)

x2=modelthermal.output

x2= GlobalAveragePooling2D()(x2)
x2= BatchNormalization()(x2)
x2= Dropout(0.5)(x2)
x2= Dense(1024,activation='relu')(x2) 
x2= Dense(128,activation='relu')(x2) 
x2= Dense(16,activation='relu')(x2) 
x2= BatchNormalization()(x2)
x2= Dropout(0.5)(x2)

# preds=Dense(8, activation='softmax')(x2) #FC-layer

In [ ]:
concat1 = concatenate([x1, x2])
Out  = Dense(8, activation='softmax')(concat1)
model_concat = Model(inputs=[modelcamera.inputs, modelthermal.inputs], outputs = Out)

In [ ]:
# model=Model(inputs=model_d.input,outputs=preds)
model_concat.summary()

In [ ]:
for model in [modelcamera, modelthermal]:
    for layer in model.layers[:-8]:
        layer.trainable=False

    for layer in model.layers[-8:]:
        layer.trainable=True

In [ ]:
model_concat.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_concat.summary()

In [ ]:
tensorflow.keras.utils.plot_model(model_concat, show_shapes=True)

In [ ]:
import glob
from PIL import Image
import tensorflow as tf
import numpy as np
from keras.utils import np_utils

# First Thermal model 

image_list = []

def my_load_Train_data_1() :
    image_list = []
    for filename in glob.glob('/root/mandeep/Data_Augmentation/Thermal Images/training/*.jpg'): #assuming gif
        im=Image.open(filename)
        img = im.resize((32,32),Image.ANTIALIAS)
        img= tf.keras.preprocessing.image.img_to_array(img)
        img= img.reshape(32, 32, 3)
        img =img.astype('float32')
        img = img / 255.0
        image_list.append(img)

    return image_list

def my_load_Test_data_1() :
    image_list = []
    for filename in glob.glob('/root/mandeep/Data_Augmentation/Thermal Images/testing/*.jpg'): #assuming gif
        im=Image.open(filename)
        img = im.resize((32,32),Image.ANTIALIAS)
        img= tf.keras.preprocessing.image.img_to_array(img)
        img= img.reshape(32, 32, 3)
        img =img.astype('float32')
        img = img / 255.0
        image_list.append(img)

    return image_list

X1_train = my_load_Train_data_1()
X1_test = my_load_Test_data_1()
#print(len(X1_test))
y1_train = pd.read_csv('/root/mandeep/Data_Augmentation/y_labels_training.csv')
y1_test = pd.read_csv('/root/mandeep/Data_Augmentation/y_labels_testing.csv')
y1_test.values.reshape(-1,1)

n_classes = 8
X1_train = np.array(X1_train)
Y1_train = np.array(y1_train)
X1_test = np.array(X1_test)
Y1_test = np.array(y1_test)

Y1_train = np_utils.to_categorical(Y1_train, n_classes)
Y1_test = np_utils.to_categorical(Y1_test, n_classes)
# len(X1_train)
# print(Y1_test)
X1_train.shape
Y1_train.shape

#Secound Camera model

image_list = []

def my_load_Train_data_2() :
    image_list = []
    for filename in glob.glob('/root/mandeep/Data_Augmentation/Camera Images/training/*.jpg'): #assuming gif
        im=Image.open(filename)
        img = im.resize((32,32),Image.ANTIALIAS)
        img= tf.keras.preprocessing.image.img_to_array(img)
        img= img.reshape(32, 32, 3)
        img =img.astype('float32')
        img = img / 255.0
        image_list.append(img)

    return image_list

def my_load_Test_data_2() :
    image_list = []
    for filename in glob.glob('/root/mandeep/Data_Augmentation/Camera Images/testing/*.jpg'): #assuming gif
        im=Image.open(filename)
        img = im.resize((32,32),Image.ANTIALIAS)
        img= tf.keras.preprocessing.image.img_to_array(img)
        img= img.reshape(32, 32, 3)
        img =img.astype('float32')
        img = img / 255.0
        image_list.append(img)

    return image_list

X2_train = my_load_Train_data_2()
X2_test = my_load_Test_data_2()
print(len(X2_test))
y2_train = pd.read_csv('/root/mandeep/Data_Augmentation/y_labels_training.csv')
y2_test = pd.read_csv('/root/mandeep/Data_Augmentation/y_labels_testing.csv')
y2_test.values.reshape(-1,1)

n_classes = 8
X2_train = np.array(X2_train)
Y2_train = np.array(y2_train)
X2_test = np.array(X2_test)
Y2_test = np.array(y2_test)

Y2_train = np_utils.to_categorical(Y2_train, n_classes)
Y2_test = np_utils.to_categorical(Y2_test, n_classes)

In [ ]:
# compiling the sequential model
model_concat.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [ ]:
history= model_concat.fit([X1_train,X2_train],Y2_train, batch_size=128,epochs=10,validation_data=([X1_test,X2_test],Y2_test))

In [ ]:
(eval_loss, eval_accuracy) = model_concat.evaluate([X1_test,X2_test],Y2_test, batch_size=128, verbose=1)
print('[INFO] Accuracy: {:.2f}%'.format(eval_accuracy * 100)) 
print('[INFO] Loss: {}'.format(eval_loss)) 